# Advent of Code 2023 - Day 3

In [110]:
import java.io.File

data class Boundary(
    val columns: IntRange,
    val rows: IntRange,
)

sealed interface Part<T> {
    val value: T
    val boundary: Boundary
}

data class NumberPart(override val value: Int, override val boundary: Boundary) : Part<Int>
data class SymbolPart(override val value: Char, override val boundary: Boundary) : Part<Char>

val NUMBER_REGEX = """(\d+)""".toRegex()
val SYMBOL_REGEX = """([^\d.])""".toRegex()

val schematic: Set<Part<*>> = File("Day03.input.txt")
    .bufferedReader()
    .lineSequence()
    .flatMapIndexed { column, line ->
        SYMBOL_REGEX.findAll(line).map {
            SymbolPart(
                value = it.value[0],
                boundary = Boundary(
                    rows = it.range,
                    columns = column..column
                )
            )
        } + NUMBER_REGEX.findAll(line).map {
            NumberPart(
                value = it.value.toInt(),
                boundary = Boundary(
                    rows = it.range,
                    columns = column..column
                )
            )
        }
    }
    .toSet()

## Part 1

Sum up all `NumberPart`s that are adjacent, including diagonally, to a `SymbolPart`.

In [111]:
infix fun Boundary.intersect(other: Boundary): Boolean =
    (rows intersect other.rows).isNotEmpty() && (columns intersect other.columns).isNotEmpty()

fun Boundary.pad(columns: Int = 0, rows: Int = 0): Boundary = Boundary(
    columns = (this.columns.start - columns)..(this.columns.endInclusive + columns),
    rows = (this.rows.start - rows)..(this.rows.endInclusive + rows),
)

schematic
    .filterIsInstance<NumberPart>()
    .filter { numberPart ->
        schematic.filterIsInstance<SymbolPart>()
            .any { numberPart.boundary.pad(1, 1) intersect it.boundary }
    }
    .sumOf(NumberPart::value)

539637

### Notes

Thinking about this problem as a collision problem I think makes this a lot easier. All you have to do is check whether two boundaries overlap.

## Part 2

A gear is any `SymbolPart` whose value is `"*"` and is adjacent to exactly two `NumberPart`s. Multiply the two `NumberPart`s to obtain the gear ratio and find the sum of all gear ratios.

In [112]:
fun SymbolPart.isGear(): Boolean = value == '*'

schematic
    .filterIsInstance<SymbolPart>()
    .filter { it.isGear() }
    .mapNotNull { gear ->
        schematic.filterIsInstance<NumberPart>()
            .filter { gear.boundary.pad(1, 1) intersect it.boundary }
            .takeIf { it.size == 2 }
            ?.let { it[0].value * it[1].value }
    }
    .sum()

82818007

### Notes

Logic is just slightly different, but the concept of treating it as a collision remains the same.